In [1]:
"""
Author: Hyewon Jeong
Last Modified: January 20, 2∂22
"""

import os
import pickle
import sys

sys.path.append("../")

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, roc_auc_score, average_precision_score, f1_score, accuracy_score, roc_curve
from sklearn.metrics import precision_recall_curve
from scipy import stats

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model

from src.hnet import AppendNet
from src.utils import do_bootstrap, confidence_interval

os.environ["CUDA_VISIBLE_DEVICES"]="3"

def load_pretrained_model(pre_trained_loc="./PCLR.h5") :
    pre_trained_model = load_model(pre_trained_loc)
    
    return pre_trained_model

2025-02-14 13:29:55.883613: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [13]:
def get_ecg(df):
    ecgs = []
    for idx in df.index:
        row = df.loc[idx]
        qid = row['QuantaID']
        doc = row['Date_of_Cath']
        fname = f'/storage/shared/apollo/same-day/{qid}_{doc}.csv'
        x = pd.read_csv(fname).values[...,1:].astype(np.float32)
        x /= 1000
        x = x[:4096, :].T
        ecgs.append(x)
        
    ecgs = np.array(ecgs)
    return np.transpose(ecgs, (0,2,1))

def get_data(batch_size=64, agegap = False):
    df_tab = pd.read_csv(os.path.join('/storage/shared/apollo/same-day/tabular_data.csv'))
    train_ids = np.load("./stores/train_ids.npy")
    val_ids = np.load("./stores/val_ids.npy")
    test_ids = np.load("./stores/test_ids.npy")
    
    train_ids = train_ids[len(train_ids) // 2 :]
    val_ids = val_ids[len(val_ids) // 2 :]
    test_ids = test_ids[len(test_ids) // 2 :]

    train_df = df_tab[df_tab["QuantaID"].isin(train_ids)]
    val_df = df_tab[df_tab["QuantaID"].isin(val_ids)]
    test_df = df_tab[df_tab["QuantaID"].isin(test_ids)]
    print(len(train_df), len(val_df), len(test_df))
        
    X_train = get_ecg(train_df)
    X_val = get_ecg(val_df)
    X_test = get_ecg(test_df)

    y_train = (train_df["PCWP_mean"].values >= 18).astype('float32')
    y_val = (val_df["PCWP_mean"].values >= 18).astype('float32')
    y_test = (test_df["PCWP_mean"].values >= 18).astype('float32')

    if agegap==True:
        age_groups = {
            1: (18, 35),
            2: (35, 50),
            3: (50, 75),
            4: (75, 100)
        }

        # Function to map age to age group
        def map_age_to_group(age):
            for group, (min_age, max_age) in age_groups.items():
                if min_age <= age < max_age:
                    return group
            return None

        # Create a new column in the dataframe to indicate the age group for each row
        test_df['Age_Group'] = test_df['Age_at_Cath'].apply(map_age_to_group)
        
        age1_df = test_df[test_df['Age_Group'] == 1]
        age2_df = test_df[test_df['Age_Group'] == 2]
        age3_df = test_df[test_df['Age_Group'] == 3]
        age4_df = test_df[test_df['Age_Group'] == 4]

        age1_test = get_ecg(age1_df)
        age2_test = get_ecg(age2_df)
        age3_test = get_ecg(age3_df)
        age4_test = get_ecg(age4_df)

        y_age1 = (age1_df["PCWP_mean"].values >= 18).astype('float32')
        y_age2 = (age2_df["PCWP_mean"].values >= 18).astype('float32')
        y_age3 = (age3_df["PCWP_mean"].values >= 18).astype('float32')
        y_age4 = (age4_df["PCWP_mean"].values >= 18).astype('float32')

        return X_train, y_train, X_val, y_val, X_test, y_test, age1_test, y_age1, age2_test, y_age2, age3_test, y_age3, age4_test, y_age4

    else:
        male_ids = np.load("./stores/test_female_ids.npy")
        female_ids = np.load("./stores/test_male_ids.npy")

        male_df = df_tab[df_tab["QuantaID"].isin(male_ids)]
        female_df = df_tab[df_tab["QuantaID"].isin(female_ids)]
        print(len(male_df), len(female_df))

        male_test = get_ecg(male_df)
        female_test = get_ecg(female_df)

        y_male = (male_df["PCWP_mean"].values >= 18).astype('float32')
        y_female = (female_df["PCWP_mean"].values >= 18).astype('float32')

        return X_train, y_train, X_val, y_val, X_test, y_test, male_test, y_male, female_test, y_female
    

In [3]:
pre_trained_model = load_pretrained_model(pre_trained_loc='./PCLR.h5')
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
full_model = AppendNet(latent, new_layers = [128, 1], classification=True)

optimizer = tf.keras.optimizers.Adam() # can modify LR, of course
loss_fn = tf.keras.losses.BinaryCrossentropy()

X_train, y_train, X_val, y_val, X_test, y_test, male_test, y_male, female_test, y_female = get_data()

2023-07-26 02:21:24.292944: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 02:21:25.753731: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 28766 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3e:00.0, compute capability: 7.0


2442 893 923
1114 711


In [4]:
epochs = 50
full_model.compile(optimizer, loss_fn)
full_model.fit(X_train, y_train, epochs=epochs)

Epoch 1/50


2023-07-26 02:22:23.653837: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-07-26 02:22:24.478271: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-07-26 02:22:24.480165: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-07-26 02:22:24.480207: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-07-26 02:22:24.482338: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-07-26 02:22:24.482469: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


77/77 [==============================] - 11s 62ms/step - loss: 1.8153
Epoch 2/50
77/77 [==============================] - 4s 55ms/step - loss: 0.8507
Epoch 3/50
77/77 [==============================] - 6s 78ms/step - loss: 0.6789
Epoch 4/50
77/77 [==============================] - 6s 82ms/step - loss: 0.6187
Epoch 5/50
77/77 [==============================] - 5s 71ms/step - loss: 0.6071
Epoch 6/50
77/77 [==============================] - 6s 74ms/step - loss: 0.6220
Epoch 7/50
77/77 [==============================] - 6s 83ms/step - loss: 0.6186
Epoch 8/50
77/77 [==============================] - 6s 75ms/step - loss: 0.6091
Epoch 9/50
77/77 [==============================] - 6s 75ms/step - loss: 0.5883
Epoch 10/50
77/77 [==============================] - 5s 66ms/step - loss: 0.5816
Epoch 11/50
77/77 [==============================] - 4s 52ms/step - loss: 0.5821
Epoch 12/50
77/77 [==============================] - 4s 54ms/step - loss: 0.5873
Epoch 13/50
77/77 [============================

In [78]:
epochs = 40
pre_trained_model = load_pretrained_model(pre_trained_loc='./PCLR.h5')
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
full_model = AppendNet(latent, new_layers = [128, 1], classification=False)

optimizer = tf.keras.optimizers.Adam() # can modify LR, of course
loss_fn = tf.keras.losses.BinaryCrossentropy()

full_model.compile(optimizer, loss_fn)
full_model.fit(X_train, y_train, epochs=epochs)

Epoch 1/40
77/77 [==============================] - 9s 56ms/step - loss: 5.6152
Epoch 2/40
77/77 [==============================] - 4s 56ms/step - loss: 5.5585
Epoch 3/40
77/77 [==============================] - 4s 54ms/step - loss: 5.5585
Epoch 4/40
77/77 [==============================] - 4s 56ms/step - loss: 5.5585
Epoch 5/40
77/77 [==============================] - 4s 54ms/step - loss: 5.5585
Epoch 6/40
77/77 [==============================] - 4s 54ms/step - loss: 5.5585
Epoch 7/40
77/77 [==============================] - 4s 55ms/step - loss: 5.5585
Epoch 8/40
77/77 [==============================] - 4s 58ms/step - loss: 5.5585
Epoch 9/40
77/77 [==============================] - 5s 63ms/step - loss: 5.5585
Epoch 10/40
77/77 [==============================] - 4s 55ms/step - loss: 5.5585
Epoch 11/40
77/77 [==============================] - 4s 55ms/step - loss: 5.5585
Epoch 12/40
77/77 [==============================] - 4s 54ms/step - loss: 5.5585
Epoch 13/40
77/77 [==================

In [75]:
epochs = 60
pre_trained_model = load_pretrained_model(pre_trained_loc='./PCLR.h5')
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
full_model = AppendNet(latent, new_layers = [128, 1], classification=False)

optimizer = tf.keras.optimizers.Adam() # can modify LR, of course
loss_fn = tf.keras.losses.BinaryCrossentropy()

full_model.compile(optimizer, loss_fn)
full_model.fit(X_train, y_train, epochs=epochs)

Epoch 1/60
77/77 [==============================] - 8s 52ms/step - loss: 5.7474
Epoch 2/60
77/77 [==============================] - 5s 59ms/step - loss: 5.3227
Epoch 3/60
77/77 [==============================] - 4s 51ms/step - loss: 5.1879
Epoch 4/60
77/77 [==============================] - 4s 54ms/step - loss: 5.2043
Epoch 5/60
77/77 [==============================] - 4s 52ms/step - loss: 5.2899
Epoch 6/60
77/77 [==============================] - 5s 62ms/step - loss: 5.0397
Epoch 7/60
77/77 [==============================] - 4s 58ms/step - loss: 5.0758
Epoch 8/60
77/77 [==============================] - 4s 56ms/step - loss: 5.0316
Epoch 9/60
77/77 [==============================] - 4s 54ms/step - loss: 4.9309
Epoch 10/60
77/77 [==============================] - 4s 56ms/step - loss: 5.0144
Epoch 11/60
77/77 [==============================] - 4s 53ms/step - loss: 4.8389
Epoch 12/60
77/77 [==============================] - 5s 61ms/step - loss: 4.9376
Epoch 13/60
77/77 [==================

In [72]:
epochs = 75
pre_trained_model = load_pretrained_model(pre_trained_loc='./PCLR.h5')
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
full_model = AppendNet(latent, new_layers = [128, 1], classification=False)

optimizer = tf.keras.optimizers.Adam() # can modify LR, of course
loss_fn = tf.keras.losses.BinaryCrossentropy()

full_model.compile(optimizer, loss_fn)
full_model.fit(X_train, y_train, epochs=epochs)

Epoch 1/75
77/77 [==============================] - 9s 54ms/step - loss: 5.5017
Epoch 2/75
77/77 [==============================] - 4s 53ms/step - loss: 6.7143
Epoch 3/75
77/77 [==============================] - 4s 54ms/step - loss: 5.8228
Epoch 4/75
77/77 [==============================] - 4s 56ms/step - loss: 5.1356
Epoch 5/75
77/77 [==============================] - 5s 59ms/step - loss: 5.2710
Epoch 6/75
77/77 [==============================] - 4s 54ms/step - loss: 5.2711
Epoch 7/75
77/77 [==============================] - 4s 55ms/step - loss: 5.1118
Epoch 8/75
77/77 [==============================] - 4s 56ms/step - loss: 5.1244
Epoch 9/75
77/77 [==============================] - 4s 54ms/step - loss: 5.1332
Epoch 10/75
77/77 [==============================] - 4s 57ms/step - loss: 4.9833
Epoch 11/75
77/77 [==============================] - 4s 54ms/step - loss: 5.0741
Epoch 12/75
77/77 [==============================] - 5s 59ms/step - loss: 5.0246
Epoch 13/75
77/77 [==================

In [69]:
epochs = 100
pre_trained_model = load_pretrained_model(pre_trained_loc='./PCLR.h5')
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
full_model = AppendNet(latent, new_layers = [128, 1], classification=False)

optimizer = tf.keras.optimizers.Adam() # can modify LR, of course
loss_fn = tf.keras.losses.BinaryCrossentropy()

full_model.compile(optimizer, loss_fn)
full_model.fit(X_train, y_train, epochs=epochs)

Epoch 1/100
77/77 [==============================] - 9s 54ms/step - loss: 5.9893
Epoch 2/100
77/77 [==============================] - 4s 54ms/step - loss: 5.5585
Epoch 3/100
77/77 [==============================] - 4s 53ms/step - loss: 5.5596
Epoch 4/100
77/77 [==============================] - 4s 58ms/step - loss: 5.5209
Epoch 5/100
77/77 [==============================] - 4s 53ms/step - loss: 7.3218
Epoch 6/100
77/77 [==============================] - 4s 55ms/step - loss: 7.7875
Epoch 7/100
77/77 [==============================] - 4s 56ms/step - loss: 7.4752
Epoch 8/100
77/77 [==============================] - 4s 53ms/step - loss: 7.8457
Epoch 9/100
77/77 [==============================] - 4s 51ms/step - loss: 7.4221
Epoch 10/100
77/77 [==============================] - 4s 54ms/step - loss: 7.7688
Epoch 11/100
77/77 [==============================] - 4s 57ms/step - loss: 7.5437
Epoch 12/100
77/77 [==============================] - 4s 55ms/step - loss: 7.5054
Epoch 13/100
77/77 [=====

In [6]:
y_pred = full_model.predict(X_test)

29/29 [==============================] - 0s 14ms/step


In [31]:
full_model.save('./PCLR_finetuned_50epc.pb', save_format='tf')

INFO:tensorflow:Assets written to: ./PCLR_finetuned_50epc.pb/assets


INFO:tensorflow:Assets written to: ./PCLR_finetuned_50epc.pb/assets


In [ ]:
# Load saved tf model
loaded_model = tf.keras.models.load_model('./PCLR_finetuned.pb')

# Calculate Classification Performance

In [80]:
'''
40 epoch performance
'''
auc, apr, acc, f1 = do_bootstrap(y_pred, y_test)
print(np.mean(auc), np.mean(apr), np.mean(acc), np.mean(f1))
print(confidence_interval(auc), confidence_interval(apr), confidence_interval(acc), confidence_interval(f1))

0.5 0.3351798483206934 0.6648201516793067 0.0
(0.5, 0.5) (0.3054983748645721, 0.36619718309859156) (0.6338028169014085, 0.6945016251354279) (0.0, 0.0)


In [7]:
'''
50 epoch performance
'''
auc, apr, acc, f1 = do_bootstrap(y_pred, y_test)
print(np.mean(auc), np.mean(apr), np.mean(acc), np.mean(f1))
print(confidence_interval(auc), confidence_interval(apr), confidence_interval(acc), confidence_interval(f1))

0.7420528515663946 0.5421483349852821 0.764765980498375 0.657019720442488
(0.710537134846387, 0.7732459181343004) (0.4917146161878393, 0.5885634218398414) (0.7356446370530878, 0.7930660888407367) (0.612448111043782, 0.697387040714995)


In [77]:
'''
60 epoch performance
'''
auc, apr, acc, f1 = do_bootstrap(y_pred, y_test)
print(np.mean(auc), np.mean(apr), np.mean(acc), np.mean(f1))
print(confidence_interval(auc), confidence_interval(apr), confidence_interval(acc), confidence_interval(f1))

0.7135456404202721 0.5147116587344185 0.7492231852654389 0.6176315236672565
(0.6807510494291622, 0.7442520302960727) (0.46605857220647273, 0.5586640078494626) (0.7204767063921993, 0.7757313109425785) (0.5704333546209287, 0.6613424879493288)


In [74]:
'''
75 epoch performance
'''
auc, apr, acc, f1 = do_bootstrap(y_pred, y_test)
print(np.mean(auc), np.mean(apr), np.mean(acc), np.mean(f1))
print(confidence_interval(auc), confidence_interval(apr), confidence_interval(acc), confidence_interval(f1))

0.6846302660676242 0.49376882366816766 0.7417410617551463 0.5698951463946115
(0.6541804733822344, 0.7145397546030037) (0.446385566822725, 0.5395325749227305) (0.71397616468039, 0.7681473456121344) (0.5187803713988198, 0.6164033145865742)


In [71]:
'''
100 epoch performance
'''
auc, apr, acc, f1 = do_bootstrap(y_pred, y_test)
print(np.mean(auc), np.mean(apr), np.mean(acc), np.mean(f1))
print(confidence_interval(auc), confidence_interval(apr), confidence_interval(acc), confidence_interval(f1))

0.6990473923899231 0.5073281407432844 0.7491733477789816 0.5933705207643143
(0.6673062791552481, 0.7294901458721476) (0.45789507885597597, 0.5547084949493752) (0.7215330444203684, 0.7757313109425785) (0.5428037459117965, 0.6393197278911565)


# Subgroup Performance: Gender

In [14]:
'''
50 epoch performance
'''
y_pred = full_model.predict(X_test)
auc, apr, acc, f1 = do_bootstrap(y_pred, y_test)
print(np.mean(auc), np.mean(apr), np.mean(acc), np.mean(f1))
print(confidence_interval(auc), confidence_interval(apr), confidence_interval(acc), confidence_interval(f1))

29/29 [==============================] - 0s 14ms/step
0.7420528515663946 0.5421483349852821 0.764765980498375 0.657019720442488
(0.710537134846387, 0.7732459181343004) (0.4917146161878393, 0.5885634218398414) (0.7356446370530878, 0.7930660888407367) (0.612448111043782, 0.697387040714995)


In [26]:
male_pred = full_model.predict(male_test)
auc_male, apr_male, acc_male, f1_male = do_bootstrap(male_pred, y_male)
print(np.mean(auc_male), np.mean(apr_male), np.mean(acc_male), np.mean(f1_male))
print(confidence_interval(auc_male), confidence_interval(apr_male), confidence_interval(acc_male), confidence_interval(f1_male))

35/35 [==============================] - 1s 18ms/step
0.7507258753965594 0.5393680476546086 0.7739066427289049 0.6610411363480164
(0.7209960206231019, 0.7786899808832602) (0.4929084617692631, 0.5817191733975555) (0.7477558348294434, 0.7998204667863554) (0.6192317920714843, 0.6986132944082641)


In [27]:
female_pred = full_model.predict(female_test)
auc_female, apr_female, acc_female, f1_female = do_bootstrap(female_pred, y_female)
print(np.mean(auc_female), np.mean(apr_female), np.mean(acc_female), np.mean(f1_female))
print(confidence_interval(auc_female), confidence_interval(apr_female), confidence_interval(acc_female), confidence_interval(f1_female))

23/23 [==============================] - 0s 14ms/step
0.7189093939892324 0.5802674796823021 0.7468917018284107 0.6454017769165924
(0.6865776801415433, 0.7523731434920649) (0.5262370606713961, 0.6316654388471022) (0.7144866385372715, 0.7777777777777778) (0.5963359956051386, 0.6918526838966204)


In [ ]:
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/auc.npy', auc)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/apr.npy', apr)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/acc.npy', acc)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/f1.npy', f1)

np.save('/storage/hyewonjeong/metricssl_02/result/pclr/auc_male.npy', auc_male)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/apr_male.npy', apr_male)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/acc_male.npy', acc_male)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/f1_male.npy', f1_male)

np.save('/storage/hyewonjeong/metricssl_02/result/pclr/auc_female.npy', auc_female)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/apr_female.npy', apr_female)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/acc_female.npy', acc_female)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/f1_female.npy', f1_female)

In [29]:
from scipy import stats

# Kruskal-Wallis test
kruskal_stat, kruskal_p_value = stats.kruskal(auc_male, auc_female)
print("Kruskal-Wallis test:")
print("  Statistic:", kruskal_stat)
print("  p-value:", kruskal_p_value)

# Independent t-test
t_stat, t_p_value = stats.ttest_ind(auc_male, auc_female)
print("\nIndependent t-test:")
print("  Statistic:", t_stat)
print("  p-value:", t_p_value)

Kruskal-Wallis test:
  Statistic: 1043.7631107089233
  p-value: 5.520066360748927e-229

Independent t-test:
  Statistic: 44.360812155488276
  p-value: 9.057658498841297e-300


In [30]:
from scipy import stats

# Kruskal-Wallis test
kruskal_stat, kruskal_p_value = stats.kruskal(apr_male, apr_female)
print("Kruskal-Wallis test:")
print("  Statistic:", kruskal_stat)
print("  p-value:", kruskal_p_value)

# Independent t-test
t_stat, t_p_value = stats.ttest_ind(apr_male, apr_female)
print("\nIndependent t-test:")
print("  Statistic:", t_stat)
print("  p-value:", t_p_value)

Kruskal-Wallis test:
  Statistic: 837.0163653948283
  p-value: 4.833496299067937e-184

Independent t-test:
  Statistic: -36.16394216400008
  p-value: 9.686798304732356e-221


In [31]:
from scipy import stats

# Kruskal-Wallis test
kruskal_stat, kruskal_p_value = stats.kruskal(f1_male, f1_female)
print("Kruskal-Wallis test:")
print("  Statistic:", kruskal_stat)
print("  p-value:", kruskal_p_value)

# Independent t-test
t_stat, t_p_value = stats.ttest_ind(f1_male, f1_female)
print("\nIndependent t-test:")
print("  Statistic:", t_stat)
print("  p-value:", t_p_value)

Kruskal-Wallis test:
  Statistic: 204.180642071072
  p-value: 2.5560427990712206e-46

Independent t-test:
  Statistic: 15.296907303573525
  p-value: 4.895397221767411e-50


In [32]:
from scipy import stats

# Kruskal-Wallis test
kruskal_stat, kruskal_p_value = stats.kruskal(acc_male, acc_female)
print("Kruskal-Wallis test:")
print("  Statistic:", kruskal_stat)
print("  p-value:", kruskal_p_value)

# Independent t-test
t_stat, t_p_value = stats.ttest_ind(acc_male, acc_female)
print("\nIndependent t-test:")
print("  Statistic:", t_stat)
print("  p-value:", t_p_value)

Kruskal-Wallis test:
  Statistic: 951.5657351536727
  p-value: 6.058867905842296e-209

Independent t-test:
  Statistic: 40.513877337497824
  p-value: 1.7923794186520421e-262


In [22]:
'''
0.7466383919640586 0.5470243498933457 0.7678234019501625 0.6629406219367823
(0.7171828525946882, 0.776655525633572) (0.4979873922374438, 0.5928792111511485) (0.7410617551462622, 0.7941495124593716) (0.6175962416107382, 0.7026278015945095)

0.764761569484173 0.5482275288751095 0.7754524236983841 0.6778789599935521
(0.7375531821039054, 0.7919339045209898) (0.5001759634677061, 0.5922628080008435) (0.7495287253141831, 0.7998204667863554) (0.6380562658083903, 0.7137246930784827)

0.7462905545013693 0.605554705155347 0.7653867791842476 0.6864282382023428
(0.7144760255433712, 0.7782598256501183) (0.5526925644888443, 0.6578182053424811) (0.7355836849507735, 0.7960618846694796) (0.6389986273922029, 0.7307709447415329)
'''


'\n0.7466383919640586 0.5470243498933457 0.7678234019501625 0.6629406219367823\n(0.7171828525946882, 0.776655525633572) (0.4979873922374438, 0.5928792111511485) (0.7410617551462622, 0.7941495124593716) (0.6175962416107382, 0.7026278015945095)\n\n0.764761569484173 0.5482275288751095 0.7754524236983841 0.6778789599935521\n(0.7375531821039054, 0.7919339045209898) (0.5001759634677061, 0.5922628080008435) (0.7495287253141831, 0.7998204667863554) (0.6380562658083903, 0.7137246930784827)\n\n0.7462905545013693 0.605554705155347 0.7653867791842476 0.6864282382023428\n(0.7144760255433712, 0.7782598256501183) (0.5526925644888443, 0.6578182053424811) (0.7355836849507735, 0.7960618846694796) (0.6389986273922029, 0.7307709447415329)\n'

# Subgroup Performance: Age

In [16]:
X_train, y_train, X_val, y_val, X_test, y_test, age1_test, y_age1, age2_test, y_age2, age3_test, y_age3, age4_test, y_age4 = get_data(agegap=True)

'''
50 epoch performance
'''
auc, apr, acc, f1 = do_bootstrap(y_pred, y_test)
print(np.mean(auc), np.mean(apr), np.mean(acc), np.mean(f1))
print(confidence_interval(auc), confidence_interval(apr), confidence_interval(acc), confidence_interval(f1))

2442 893 923


/tmp/ipykernel_871515/3927646973.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Age_Group'] = test_df['Age_at_Cath'].apply(map_age_to_group)


0.7420528515663946 0.5421483349852821 0.764765980498375 0.657019720442488
(0.710537134846387, 0.7732459181343004) (0.4917146161878393, 0.5885634218398414) (0.7356446370530878, 0.7930660888407367) (0.612448111043782, 0.697387040714995)


In [19]:
print('========================== Age1 (18 <= age < 35) ==========================')
age1_pred = full_model.predict(age1_test)
auc_age1, apr_age1, acc_age1, f1_age1 = do_bootstrap(age1_pred, y_age1)
print(np.mean(auc_age1), np.mean(apr_age1), np.mean(acc_age1), np.mean(f1_age1))
print(confidence_interval(auc_age1), confidence_interval(apr_age1), confidence_interval(acc_age1), confidence_interval(f1_age1))

print('========================== Age2 (35 <= age < 50) ==========================')
age2_pred = full_model.predict(age2_test)
auc_age2, apr_age2, acc_age2, f1_age2 = do_bootstrap(age2_pred, y_age2)
print(np.mean(auc_age2), np.mean(apr_age2), np.mean(acc_age2), np.mean(f1_age2))
print(confidence_interval(auc_age2), confidence_interval(apr_age2), confidence_interval(acc_age2), confidence_interval(f1_age2))

print('========================== Age3 (50 <= age < 75) ==========================')
age3_pred = full_model.predict(age3_test)
auc_age3, apr_age3, acc_age3, f1_age3 = do_bootstrap(age3_pred, y_age3)
print(np.mean(auc_age3), np.mean(apr_age3), np.mean(acc_age3), np.mean(f1_age3))
print(confidence_interval(auc_age3), confidence_interval(apr_age3), confidence_interval(acc_age3), confidence_interval(f1_age3))

print('========================== Age4 (75 <= age) ==========================')
age4_pred = full_model.predict(age4_test)
auc_age4, apr_age4, acc_age4, f1_age4 = do_bootstrap(age4_pred, y_age4)
print(np.mean(auc_age4), np.mean(apr_age4), np.mean(acc_age4), np.mean(f1_age4))
print(confidence_interval(auc_age4), confidence_interval(apr_age4), confidence_interval(acc_age4), confidence_interval(f1_age4))

========================== Age1 (18 <= age < 35) ==========================
1/1 [==============================] - 0s 33ms/step
0.6557375688180642 0.31531872176480025 0.6590299277605778 0.4114250171336408
(0.286904761904762, 0.9166666666666667) (0.06666666666666667, 0.6666666666666666) (0.4, 0.8666666666666667) (0.0, 0.7938461538461524)
========================== Age2 (35 <= age < 50) ==========================
4/4 [==============================] - 0s 58ms/step
0.7306326548666713 0.5721143619809684 0.7915523809523811 0.6330675336039853
(0.6421535440301667, 0.8226086981611695) (0.4239908595016954, 0.7216182249322493) (0.7142857142857143, 0.8666666666666667) (0.4814444444444445, 0.7647549019607843)
========================== Age3 (50 <= age < 75) ==========================
21/21 [==============================] - 0s 17ms/step
0.7488592356617341 0.5170684306259999 0.7923333333333333 0.6425974855004549
(0.710632676147382, 0.7861268135024323) (0.452750956995287, 0.5838028749172977) (0.7601

In [21]:
def stat_testing(list1, list2, list3=None, list4=None):
    if list3 is not None:
        kruskal_stat, kruskal_p_value = stats.kruskal(list1, list2, list3, list4)
        t_stat, t_p_value = stats.ttest_ind(list1, list2)
        return kruskal_stat, kruskal_p_value, t_stat, t_p_value
    
    else: # age performance gap
        kruskal_stat, kruskal_p_value = stats.kruskal(list1, list2, list3, list4)
        anova_stat, anova_p_value = f_oneway(list1, list2, list3, list4)
        return kruskal_stat, kruskal_p_value, anova_stat, anova_p_value


kruskal_stat, kruskal_p_value, anova_stat, anova_p_value = stat_testing(auc_age1, auc_age2, auc_age3, auc_age4)
print("Kruskal-Wallis test for AUC: statistics {} pvalue {}".format(kruskal_stat, kruskal_p_value))
print("\nIndependent t-test for AUC: statistics {} pvalue {}".format(anova_stat, anova_p_value))

kruskal_stat, kruskal_p_value, t_stat, t_p_value = stat_testing(apr_age1, apr_age2, apr_age3, apr_age4)
print("Kruskal-Wallis test for APR: statistics {} pvalue {}".format(kruskal_stat, kruskal_p_value))
print("\nOne-way ANOVA test for APR: statistics {} pvalue {}".format(anova_stat, anova_p_value))

Kruskal-Wallis test for AUC: statistics 1403.9363143006196 pvalue 4.121218625370432e-304

Independent t-test for AUC: statistics -13.388198473606396 pvalue 3.482888404522028e-39
Kruskal-Wallis test for APR: statistics 2213.458261568214 pvalue 0.0

One-way ANOVA test for APR: statistics -13.388198473606396 pvalue 3.482888404522028e-39


In [22]:
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/auc.npy', auc)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/apr.npy', apr)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/acc.npy', acc)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/f1.npy', f1)

np.save('/storage/hyewonjeong/metricssl_02/result/pclr/auc_age1.npy', auc_age1)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/apr_age1.npy', apr_age1)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/acc_age1.npy', acc_age1)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/f1_age1.npy', f1_age1)

np.save('/storage/hyewonjeong/metricssl_02/result/pclr/auc_age2.npy', auc_age2)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/apr_age2.npy', apr_age2)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/acc_age2.npy', acc_age2)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/f1_age2.npy', f1_age2)

np.save('/storage/hyewonjeong/metricssl_02/result/pclr/auc_age3.npy', auc_age3)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/apr_age3.npy', apr_age3)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/acc_age3.npy', acc_age3)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/f1_age3.npy', f1_age3)

np.save('/storage/hyewonjeong/metricssl_02/result/pclr/auc_age4.npy', auc_age4)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/apr_age4.npy', apr_age4)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/acc_age4.npy', acc_age4)
np.save('/storage/hyewonjeong/metricssl_02/result/pclr/f1_age4.npy', f1_age4)

In [29]:
# List of age groups and corresponding lists
age_groups = ['age1', 'age2', 'age3', 'age4']
auc_lists = [np.mean(auc_age1), np.mean(auc_age2), np.mean(auc_age3), np.mean(auc_age4)]
apr_lists = [np.mean(apr_age1), np.mean(apr_age2), np.mean(apr_age3), np.mean(apr_age4)]
# rmse_lists = [age1_rmse_list, age2_rmse_list, age3_rmse_list, age4_rmse_list]

# Dictionary to store pairwise performance gaps
pairwise_gaps = {}

# Calculate pairwise performance gaps for auc, apr, and rmse
for i in range(len(age_groups)):
    for j in range(i+1, len(age_groups)):
        gap_auc = abs(auc_lists[i] - auc_lists[j])
        gap_apr = abs(apr_lists[i] - apr_lists[j])
#         gap_rmse = np.mean(np.abs(np.array(rmse_lists[i]) - np.array(rmse_lists[j])))
        key = f"{age_groups[i]} - {age_groups[j]}"
        pairwise_gaps[key] = {'AUC Gap': gap_auc, 'APR Gap': gap_apr} #, 'RMSE Gap': gap_rmse}

# Calculate average of the pairwise performance gaps
avg_gap_auc = np.mean([pairwise_gaps[key]['AUC Gap'] for key in pairwise_gaps])
avg_gap_apr = np.mean([pairwise_gaps[key]['APR Gap'] for key in pairwise_gaps])
# avg_gap_rmse = np.mean([pairwise_gaps[key]['RMSE Gap'] for key in pairwise_gaps])

# Print the results
print("Pairwise Performance Gaps:")
for key, gaps in pairwise_gaps.items():
    print(f"{key}: {gaps}")

print("\nAverage of Pairwise Performance Gaps:")
print(f"AUC Gap: {avg_gap_auc:.4f}")
print(f"APR Gap: {avg_gap_apr:.4f}")
# print(f"RMSE Gap: {avg_gap_rmse:.4f}")

Pairwise Performance Gaps:
age1 - age2: {'AUC Gap': 0.07489508604860706, 'APR Gap': 0.2567956402161682}
age1 - age3: {'AUC Gap': 0.09312166684366985, 'APR Gap': 0.2017497088611997}
age1 - age4: {'AUC Gap': 0.017313655509358594, 'APR Gap': 0.29601615038762064}
age2 - age3: {'AUC Gap': 0.018226580795062786, 'APR Gap': 0.05504593135496849}
age2 - age4: {'AUC Gap': 0.09220874155796566, 'APR Gap': 0.03922051017145245}
age3 - age4: {'AUC Gap': 0.11043532235302844, 'APR Gap': 0.09426644152642094}

Average of Pairwise Performance Gaps:
AUC Gap: 0.0677
APR Gap: 0.1572
